### Anaconda & Python Version:

The version of the notebook server is 5.0.0 and is running on:
Python 3.6.4 |Anaconda custom (64-bit)| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]

Current Kernel Information:
Python 3.6.4 |Anaconda custom (64-bit)| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]
Type 'copyright', 'credits' or 'license' for more information
IPython 6.1.0 -- An enhanced Interactive Python. Type '?' for help.

#### Anaconda 5.0.1

### Steps:
#### 0. Drop unnecessary variables like Id, Name
#### 1. Dummy Coding
#### 2. Train-Validation-Test Split (2 Validation sets, 1 Test set)
#### 3. Separate X and Y of all datasets & convert to array/matrix
#### 4. Mean Standardization
#### 5. Create basic ANN & validation & testing
#### 6. ANN Parameter Tuning - 4 levels (includes: Avoiding Overfitting(Dropout Regularization)
#### 7. Download and Upload Model

#### Summary Notes:

#### Code Cheat Sheet NN - https://www.datacamp.com/community/blog/keras-cheat-sheet

In [1]:
# Python Version: Python 3.6.4 |Anaconda custom (64-bit)| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]
# Anaconda 5.0.1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn # same as scikit-learn
df1 = pd.read_csv('C:/Users/anurag.trivedi/2. Data/Churn_Modelling.csv')
print(df1)
# Installing Theano, Tensorflow, Keras
# Open anaconda command line:
# 'pip/conda install theano' # pip/conda install theano==1.0.1
# 'pip/conda install tensorflow' # for CPU and not GPU # pip/conda install tensorflow==1.4.0
# 'conda install keras' # conda install keras==2.1.2
# 'conda update --all' # or can update specific library by 'conda update keras'

# 'conda info keras==2.1.2' # to check dependencies of a package

      RowNumber  CustomerId         Surname  CreditScore Geography  Gender  \
0             1    15634602        Hargrave          619    France  Female   
1             2    15647311            Hill          608     Spain  Female   
2             3    15619304            Onio          502    France  Female   
3             4    15701354            Boni          699    France  Female   
4             5    15737888        Mitchell          850     Spain  Female   
5             6    15574012             Chu          645     Spain    Male   
6             7    15592531        Bartlett          822    France    Male   
7             8    15656148          Obinna          376   Germany  Female   
8             9    15792365              He          501    France    Male   
9            10    15592389              H?          684    France    Male   
10           11    15767821          Bearce          528    France    Male   
11           12    15737173         Andrews          497     Spa

In [2]:
# theano & tensorflow are libraries for faster ANN computations on GPUs
# keras is wrapper library on top of theano/tensorflow which is intuitive and easy to use

In [3]:
# Versions of Libraries: (Code: 'conda list'/'pip list' in anaconda prompt)
# PIP Version - 10.0.1
# conda Version - 4.5.11 # do this first
# pandas (0.23.4)
# numpy (1.15.2) # python -m pip install numpy==1.12.1
# matplotlib (3.0.0)
# scikit-learn (0.19.1)
# scipy (0.1.0)
# Theano (1.0.3)
# tensorflow (1.11.0) # how to change version: https://stackoverflow.com/questions/45749992/how-to-downgrade-tensorflow-multiple-versions-possible
# keras (2.2.4)
# torch (0.4.1) # installed in Boltzmann Machines code
# python (3.6.6)

# Check version of anaconda - 'conda list anaconda$' or or 'conda info' in anaconda command prompt
# Check version of python - 'python' in anaconda command prompt

# Uninstallation:
# run 'conda list' # some packages will have 'pip' written against them, these can be unistalled only by pip
# 'conda uninstall tensorflow==1.4.0' or 'conda uninstall tensorflow==1.4.0'

### 0. Drop unnecessary variables like Id, Name

In [4]:
###########################################################
# 0. Drop unnecessary variables like Id, Name
###########################################################
df1 = df1.drop(['RowNumber','CustomerId','Surname'], axis=1)

### 1. Dummy Coding

In [5]:
###########################################################
# 1. Dummy Coding
###########################################################
df2 = df1.copy()
df2_new = df2.copy()
for i in range(len(df2.columns)):
        if (df2.iloc[:,i].dtype==np.object or df2.iloc[:,i].dtype.name=="category"):
            df2_dummy = pd.get_dummies(df2.iloc[:,i]) # ignores NA
            for j in range(len(df2_dummy.columns)):
                df2_dummy.columns.values[j] = df2.columns.values[i] + "_" + df2_dummy.columns.values[j]
            df2_new = pd.concat([df2_new, df2_dummy], axis=1)
for y in df2_new.columns: # new vars created are of type ‘uint8’, so converting to int
     if(df2_new [y].dtype == np.uint8):
        df2_new [y] = df2_new [y].astype('int')
        
dfx = df2_new.copy()
S_List_Numeric = []
S_List_Non_Numeric = []
for i in range(len(dfx.columns)):
    if (dfx.iloc[:,i].dtype==np.int64 or dfx.iloc[:,i].dtype==np.float64 or dfx.iloc[:,i].dtype==np.int32 or dfx.iloc[:,i].dtype==np.float32 or dfx.iloc[:,i].dtype==np.int16 or dfx.iloc[:,i].dtype==np.float16):
        S_List_Numeric.append(i)
    else:
        S_List_Non_Numeric.append(i)
dfx_Numeric = dfx.iloc[:,S_List_Numeric]
     
df2 = dfx_Numeric.copy()
print(df2)

      CreditScore  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
0             619   42       2       0.00              1          1   
1             608   41       1   83807.86              1          0   
2             502   42       8  159660.80              3          1   
3             699   39       1       0.00              2          0   
4             850   43       2  125510.82              1          1   
5             645   44       8  113755.78              2          1   
6             822   50       7       0.00              2          1   
7             376   29       4  115046.74              4          1   
8             501   44       4  142051.07              2          0   
9             684   27       2  134603.88              1          1   
10            528   31       6  102016.72              2          0   
11            497   24       3       0.00              2          1   
12            476   34      10       0.00              2          1   
13    

In [6]:
###########################################################
# drop last dummy coded variable from each categorical variable: here, Geography_Spain and Gender_Male
###########################################################
df2 = df2.drop(['Geography_Spain','Gender_Male'], axis=1) 
df2.dtypes

CreditScore            int64
Age                    int64
Tenure                 int64
Balance              float64
NumOfProducts          int64
HasCrCard              int64
IsActiveMember         int64
EstimatedSalary      float64
Exited                 int64
Geography_France       int32
Geography_Germany      int32
Gender_Female          int32
dtype: object

### 2. Train-Validation Split (2 Validation sets)

In [7]:
###########################################################
# 2. Train-Validation Split (2 Validation sets)
###########################################################
np.random.seed(4)

msk = np.random.rand(len(df2)) < 0.7
Train = df2[msk] # generates random sample of nearly 70%
Temp = df2[~msk]

msk = np.random.rand(len(Temp)) < 0.33
Vald1 = Temp[msk] # generates random sample of nearly 10%
Temp1 = Temp[~msk]

msk = np.random.rand(len(Temp1)) < 0.50
Vald2 = Temp1[msk] # generates random sample of nearly 10%
Test = Temp1[~msk] # generates random sample of nearly 10%

# Train, Vald1, Vald2 and Test are ready
print(Train)
print(Test)

      CreditScore  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
1             608   41       1   83807.86              1          0   
4             850   43       2  125510.82              1          1   
5             645   44       8  113755.78              2          1   
7             376   29       4  115046.74              4          1   
8             501   44       4  142051.07              2          0   
9             684   27       2  134603.88              1          1   
11            497   24       3       0.00              2          1   
14            635   35       7       0.00              2          1   
15            616   45       3  143129.41              2          0   
16            653   58       1  132602.88              1          1   
17            549   24       9       0.00              2          1   
18            587   45       6       0.00              1          0   
20            732   41       8       0.00              2          1   
24    

### 3. Separate X and Y of all datasets

In [8]:
###########################################################
# 3. Separate X and Y of all datasets
###########################################################
# Convert all X and Y into array/matrix form for model building

X_Train = Train.drop(['Exited'], axis=1)
Y_Train = Train[["Exited"]]
X_Train=X_Train.as_matrix()
Y_Train=Y_Train.as_matrix()

X_Vald1 = Vald1.drop(['Exited'], axis=1)
Y_Vald1 = Vald1[["Exited"]]
X_Vald1=X_Vald1.as_matrix()
Y_Vald1=Y_Vald1.as_matrix()

X_Vald2 = Vald2.drop(['Exited'], axis=1)
Y_Vald2 = Vald2[["Exited"]]
X_Vald2=X_Vald2.as_matrix()
Y_Vald2=Y_Vald2.as_matrix()

X_Test = Test.drop(['Exited'], axis=1)
Y_Test = Test[["Exited"]]
X_Test=X_Test.as_matrix()
Y_Test=Y_Test.as_matrix()

C:\Users\anurag.trivedi\Anaconda1\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\anurag.trivedi\Anaconda1\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':
C:\Users\anurag.trivedi\Anaconda1\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]
C:\Users\anurag.trivedi\Anaconda1\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\anurag.trivedi\Anaconda1\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\anurag.trivedi\Anaconda1\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: Method .as_

### 4. Mean Standardization

In [9]:
###########################################################
# 4. Mean Standardization
###########################################################

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_Train = sc.fit_transform(X_Train)
X_Vald1 = sc.transform(X_Vald1)
X_Vald2 = sc.transform(X_Vald2)
X_Test = sc.transform(X_Test)

### 5. Create basic ANN & validation & testing

In [10]:
###########################################################
# 5. Create basic ANN & validation & testing
###########################################################
# Keras Documentation: https://keras.io/getting-started/sequential-model-guide/

# importing keras libraries and packages:
import keras
from keras.models import Sequential # to initialize the ANN
from keras.layers import Dense # to create layers of ANN

Using TensorFlow backend.


In [11]:
# Creating Basic ANN
# Good Link for binary, categrical, regression ANN: https://www.datacamp.com/community/blog/keras-cheat-sheet

np.random.seed(100) # works. gives close results on repeating but not exactly same.

# Initializing ANN
classifier = Sequential()

# Adding input layer and 1st hidden layer
# classifier.add(Dense(output_dim=6, init="uniform", activation="relu", input_dim=12)) # suggested by course
classifier.add(Dense(activation="relu", input_dim=11, units=55, kernel_initializer="uniform")) # suggested by keras warning
# 'output_dim'/'units' is number of hidden neurons in 1st hidden layer
# 'init'/'kernel_initializer' specifies how to initialize weights. 'uniform' ensures close to 0 and uniform values for initialization
# activation="relu" is 'rectifier' activation function for hidden layer; which is considered most impactful for hidden layers
# 'input_dim' is number of input layer neurons which is equal to number of input variables
# code fr multi-class: https://www.datacamp.com/community/blog/keras-cheat-sheet
# 'relu' is good for Regression also

# Adding 2nd hidden layer
classifier.add(Dense(activation="relu", units=55, kernel_initializer="uniform"))
# as per instructor, generally 2 layers are sufficient for normal data problems

# # Adding hidden layer
# classifier.add(Dense(activation="relu", units=70, kernel_initializer="uniform"))
# # Adding hidden layer
# classifier.add(Dense(activation="relu", units=7, kernel_initializer="uniform"))
# # Adding hidden layer
# classifier.add(Dense(activation="relu", units=7, kernel_initializer="uniform"))
# # Adding hidden layer
# classifier.add(Dense(activation="relu", units=35, kernel_initializer="uniform"))
# Adding hidden layer
# classifier.add(Dense(activation="relu", units=35, kernel_initializer="uniform"))
# # Adding hidden layer
# classifier.add(Dense(activation="relu", units=35, kernel_initializer="uniform"))
# # Adding hidden layer
# classifier.add(Dense(activation="relu", units=35, kernel_initializer="uniform"))

# Adding output layer
classifier.add(Dense(activation="sigmoid", units=1, kernel_initializer="uniform"))
# for binary outcome: units=1 and activation="sigmoid"
# for categorical(say 3) outcome: units=3 and activation="softmax"
# for regression, 'model.add(Dense(1))' is the output layer

# Compiling the ANN (means run SGD on ANN)
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# compiling means applying opt algo on ANN
# for binary classification, loss function 'binary_crossentropy'(='log loss'); 
    #for >2, it's 'categorical_crossentropy'
# for multi-class classification, datascamp suggests 'rmsprop' as optimizer
# for regression: ' model.compile(optimizer='rmsprop',loss='mse',metrics=['mae'])) ' is the sample code
# Documentation on optimizer: https://keras.io/optimizers/

# Fitting ANN to training set
classifier.fit(X_Train, Y_Train, batch_size=10, epochs=100)

Epoch 1/100
7014/7014 [==============================] - 1s 190us/step - loss: 0.4474 - acc: 0.7991
Epoch 2/100
7014/7014 [==============================] - 1s 128us/step - loss: 0.4177 - acc: 0.8293
Epoch 3/100
7014/7014 [==============================] - 1s 102us/step - loss: 0.4080 - acc: 0.8353
Epoch 4/100
7014/7014 [==============================] - 1s 102us/step - loss: 0.3981 - acc: 0.8403
Epoch 5/100
7014/7014 [==============================] - 1s 105us/step - loss: 0.3766 - acc: 0.8499
Epoch 6/100
7014/7014 [==============================] - 1s 119us/step - loss: 0.3539 - acc: 0.8581
Epoch 7/100
7014/7014 [==============================] - 1s 110us/step - loss: 0.3444 - acc: 0.8597
Epoch 8/100
7014/7014 [==============================] - 1s 143us/step - loss: 0.3421 - acc: 0.8611
Epoch 9/100
7014/7014 [==============================] - 1s 147us/step - loss: 0.3393 - acc: 0.8621
Epoch 10/100
7014/7014 [==============================] - 1s 108us/step - loss: 0.3372 - acc: 0.8636

7014/7014 [==============================] - 1s 91us/step - loss: 0.2620 - acc: 0.8958
Epoch 83/100
7014/7014 [==============================] - 1s 90us/step - loss: 0.2633 - acc: 0.8934
Epoch 84/100
7014/7014 [==============================] - 1s 90us/step - loss: 0.2600 - acc: 0.8942
Epoch 85/100
7014/7014 [==============================] - 1s 105us/step - loss: 0.2586 - acc: 0.8949
Epoch 86/100
7014/7014 [==============================] - 1s 95us/step - loss: 0.2602 - acc: 0.8944
Epoch 87/100
7014/7014 [==============================] - 1s 87us/step - loss: 0.2580 - acc: 0.8969
Epoch 88/100
7014/7014 [==============================] - 1s 89us/step - loss: 0.2570 - acc: 0.8978
Epoch 89/100
7014/7014 [==============================] - 1s 92us/step - loss: 0.2559 - acc: 0.8989
Epoch 90/100
7014/7014 [==============================] - 1s 89us/step - loss: 0.2578 - acc: 0.8921
Epoch 91/100
7014/7014 [==============================] - 1s 88us/step - loss: 0.2554 - acc: 0.8936
Epoch 92/100

In [12]:
# Model Validation
# Calculating auc for Vald1 and Vald2 sets (also has code for 'F1_Score' below)

# select best classifiers among 'classifier_best', 'classifier_best_1', 'classifier_best_2'
# depending on which tuning level above gave best model
# classifier = classifier_best

# Predict for Vald1 set
y_pred = classifier.predict(X_Vald1)
y_pred = (y_pred > 0.5)

# creating CF matrix
from sklearn.metrics import confusion_matrix
cm1 = confusion_matrix(y_pred, Y_Vald1)

# calculating auc
from sklearn.metrics import roc_curve, auc, roc_auc_score
fp_rate, tp_rate, thresholds = roc_curve(Y_Vald1, y_pred)
print(auc(fp_rate, tp_rate))

# printing CF
## cm

###########################################################

# Predict for Vald2 set
y_pred = classifier.predict(X_Vald2)
y_pred = (y_pred > 0.5)

# creating CF matrix
from sklearn.metrics import confusion_matrix
cm2 = confusion_matrix(y_pred, Y_Vald2)

# calculating auc
from sklearn.metrics import roc_curve, auc, roc_auc_score
fp_rate, tp_rate, thresholds = roc_curve(Y_Vald2, y_pred)
print(auc(fp_rate, tp_rate))

# printing CF
## cm

###########################################################



###########################################################
# Code for F1_Score:
###########################################################


# from sklearn.metrics import precision_recall_fscore_support

# y_pred = classifier.predict(X_Vald1)
# y_pred = (y_pred > 0.5)

# precision_recall_fscore_support(Y_Vald1, y_pred, average=None, pos_label=1)
# F1 = precision_recall_fscore_support(Y_Vald1, y_pred, average=None, pos_label=1)
# F1_Score = 2 * F1[0][1] * F1[1][1]/(F1[0][1] + F1[1][1])
# print(F1_Score)

# ###########################################################

# y_pred = classifier.predict(X_Vald2)
# y_pred = (y_pred > 0.5)

# precision_recall_fscore_support(Y_Vald2, y_pred, average=None, pos_label=1)
# F1 = precision_recall_fscore_support(Y_Vald2, y_pred, average=None, pos_label=1)
# F1_Score = 2 * F1[0][1] * F1[1][1]/(F1[0][1] + F1[1][1])
# print(F1_Score)

# ###########################################################

0.7323919474001486
0.7159383033419024


### 6. ANN Parameter Tuning - 4 Levels

In [42]:
###########################################################
# 6. ANN Tuning - 4 levels
###########################################################
###################################### ANN Parameter Tuning - Manual (** are mainly impactful)
# 1. **Number of layers (default: 2). Increasing beyond 2 gave good results
# 2. **Number of neurons in each layer (default: mean_neurons 'input layer neurons + output layer neurons').  
#    Increasing neurons to much above the mean_neurons gives good results.
#    In this case, mean was 6 but increasing to 35 gave good results
# 4. **batch_size=10, **epochs=30
# 5. *optimizers = ['rmsprop', 'adam'] # optimizer 'adam' gave better accuracy than 'rmsprop'
# 6. *init = ['glorot_uniform', 'normal', 'uniform']

# Above example best result:
# 'relu' and 'sigmoid' in hidden layers and output layer respectively
# 'adam' optimizer
# batch_size=10, epochs=30
# parameters selected only on basis of vald1 and vald2 sets
# layers=2, neurons=70 each -> gave 0.71 auc on vald and test; adding 3rd layer with 70 neurons reduced accuracy;
# * layers=2, neurons=120 each -> accuracy increased to 0.7248, 0.7171 for vald and 0.7169 for test
# layers=2, neurons=150 each -> accuracy reduced to 0.72, 0.69 for vald and 0.70 for test
# layers=2, neurons=110 each -> accuracy increased to 0.7286, 0.7296 for vald and 0.7116 for test
# layers=2, neurons=100 each -> accuracy reduced to 0.7035, 0.7062 for vald and 0.7238 for test
# layers=2, neurons=115 each -> accuracy reduced to 0.7109, 0.7056 for vald and 0.7233 for test

# Summary: Manual tuning leads to lower accuracy since you cannot check 100s or 1000s of iterations

###########################################################
###################################### ANN Parameter Tuning - Level 1
###################################### Best 'number of layers' and 'neurons' in a layer
###########################################################
# Using for loop for parameter tuning
# Strategy: Tune parameters sequentially to save time
# 1st - Number of layers and neurons
# 2nd - batch size and epoch

np.random.seed(100)

auc1=0
auc2=0
best_auc1=auc1
best_auc2=auc2
best_layers = 0
best_units = 0

for i in range(2,3): # number of hidden layers(over and above the 1 default hidden layer); normal problems-best results at 2
    for j in range(5,130,5): # number of neurons in each hidden layer
        classifier = Sequential()
        classifier.add(Dense(activation="relu", input_dim=11, units=j, kernel_initializer="uniform"))
        for k in range(1,i): # adding hidden layers dynamically
            classifier.add(Dense(activation="relu", units=j, kernel_initializer="uniform"))
        classifier.add(Dense(activation="sigmoid", units=1, kernel_initializer="uniform"))
        classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        classifier.fit(X_Train, Y_Train, batch_size=10, epochs=100)
        
        y_pred = classifier.predict(X_Vald1)
        y_pred = (y_pred > 0.5)
        fp_rate, tp_rate, thresholds = roc_curve(Y_Vald1, y_pred)
        auc1 = auc(fp_rate, tp_rate)
        
        y_pred = classifier.predict(X_Vald2)
        y_pred = (y_pred > 0.5)
        fp_rate, tp_rate, thresholds = roc_curve(Y_Vald2, y_pred)
        auc2 = auc(fp_rate, tp_rate)
    
        # check code progress
        print("current_auc1: ", auc1) # to check ongoing current auc1
        print("current_auc2: ", auc2) # to check ongoing current auc2
        print("current_layers: ", i) # to check ongoing current hidden layers
        print("current_units: ", j) # to check ongoing current units
    
        if (auc1 > best_auc1 and auc2 > best_auc2):
            best_auc1 = auc1
            best_auc2 = auc2
            best_layers = i
            best_units = j
            classifier_best = classifier # saving the best model
        
        # check ongoing best parameters during execution
        print("best_auc1: ", best_auc1) # to check ongoing best auc1
        print("best_auc2: ", best_auc2) # to check ongoing best auc2
        print("best_layers: ", best_layers) # to check ongoing best auc2
        print("best_units: ", best_units) # to check ongoing best best_units
    
print("final_best_auc1: ", best_auc1)
print("final_best_auc2: ", best_auc2)
print("final_best_layers: ", best_layers)
print("final_best_units: ", best_units)

classifier = classifier_best # saving final model as 'classifier' to predict

# final_best_auc1:  0.715288584206
# final_best_auc2:  0.714950923113
# final_best_layers:  2
# final_best_units:  55

# Summary: Accuracy improved at this tuning level

Epoch 1/100
7014/7014 [==============================] - 1s 182us/step - loss: 0.4959 - acc: 0.7990
Epoch 2/100
7014/7014 [==============================] - 1s 148us/step - loss: 0.4330 - acc: 0.7993
Epoch 3/100
7014/7014 [==============================] - 1s 142us/step - loss: 0.4288 - acc: 0.7993
Epoch 4/100
4540/7014 [==================>...........] - ETA: 0s - loss: 0.4273 - acc: 0.7969

KeyboardInterrupt: 

In [343]:
###########################################################
############################### ANN Tuning - Level 2
############################### Best number of 'batch_size' and 'epochs'
###########################################################

# tuning for 'batch_size' and 'epochs'
# use number of layers and number of neurons from previously executed tuning

np.random.seed(100)

############################### Best number of 'batch_size' and 'epochs'
auc1=0
auc2=0
best_auc1=best_auc1 # best auc of 1st level tuning is base accuracy at 2nd level
best_auc2=best_auc2 # best auc of 1st level tuning is base accuracy at 2nd level
best_layers = best_layers # taken from 1st level tuning
best_units = best_units # taken from 1st level tuning 
best_batch_size = 10 # taken from 1st level tuning(fixed in 1st level)
best_epochs = 100 # taken from 1st level tuning(fixed in 1st level)

for i in range(200,300,5): # batch_size range (generally used: 10)
    for j in range(50,500,25): # epochs range (generally used: 100)
        classifier = Sequential()
        classifier.add(Dense(activation="relu", input_dim=11, units=best_units, kernel_initializer="uniform"))
        for k in range(1,best_layers): # adding hidden layers dynamically
            classifier.add(Dense(activation="relu", units=best_units, kernel_initializer="uniform"))
        classifier.add(Dense(activation="sigmoid", units=1, kernel_initializer="uniform"))
        classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        classifier.fit(X_Train, Y_Train, batch_size=i, epochs=j)
        
        y_pred = classifier.predict(X_Vald1)
        y_pred = (y_pred > 0.5)
        fp_rate, tp_rate, thresholds = roc_curve(Y_Vald1, y_pred)
        auc1 = auc(fp_rate, tp_rate)
        
        y_pred = classifier.predict(X_Vald2)
        y_pred = (y_pred > 0.5)
        fp_rate, tp_rate, thresholds = roc_curve(Y_Vald2, y_pred)
        auc2 = auc(fp_rate, tp_rate)
        
        # check code progress
        print("current_auc1: ", auc1) # to check ongoing current auc1
        print("current_auc2: ", auc2) # to check ongoing current auc2
        print("current_batch_size: ", i) # to check ongoing current batch_size
        print("current_epochs: ", j) # to check ongoing current epochs
    
        if (auc1 > best_auc1 and auc2 > best_auc2):
            best_auc1 = auc1
            best_auc2 = auc2
            best_batch_size = i
            best_epochs = j
            classifier_best_1 = classifier # saving the best model
        
        # check ongoing best parameters during execution
        print("best_auc1: ", best_auc1) # to check ongoing best auc1
        print("best_auc2: ", best_auc2) # to check ongoing best auc2
        print("best_batch_size: ", best_batch_size) # to check ongoing best auc2
        print("best_epochs: ", best_epochs) # to check ongoing best best_units
    
print("final_best_auc1: ", best_auc1)
print("final_best_auc2: ", best_auc2)
print("final_best_batch_size: ", best_batch_size)
print("final_best_epochs: ", best_epochs)

classifier = classifier_best_1 # saving final model as 'classifier' to predict

# Note: did not give better result than 1st level tuning, so model obtained during 1st level is final

# final_best_auc1:  0.715288584206
# final_best_auc2:  0.714950923113
# final_best_batch_size:  10
# final_best_epochs:  100

# Summary: Accuracy did not improve at this tuning level

Epoch 1/50
7053/7053 [==============================] - 8s 1ms/step - loss: 0.6456 - acc: 0.7961
Epoch 2/50
7053/7053 [==============================] - 0s 35us/step - loss: 0.4722 - acc: 0.7972
Epoch 3/50
7053/7053 [==============================] - 0s 34us/step - loss: 0.4384 - acc: 0.7972
Epoch 4/50
7053/7053 [==============================] - 0s 33us/step - loss: 0.4326 - acc: 0.7972
Epoch 5/50
7053/7053 [==============================] - 0s 37us/step - loss: 0.4288 - acc: 0.7972
Epoch 6/50
7053/7053 [==============================] - 0s 38us/step - loss: 0.4252 - acc: 0.7972
Epoch 7/50
7053/7053 [==============================] - 0s 40us/step - loss: 0.4219 - acc: 0.8049
Epoch 8/50
7053/7053 [==============================] - 0s 41us/step - loss: 0.4180 - acc: 0.8172
Epoch 9/50
7053/7053 [==============================] - 0s 41us/step - loss: 0.4140 - acc: 0.8272
Epoch 10/50
7053/7053 [==============================] - 0s 41us/step - loss: 0.4105 - acc: 0.8318
Epoch 11/50
7053/705

KeyboardInterrupt: 

In [353]:
###########################################################
############################### ANN Tuning - Level 3
############################### Dropout Regularization - avoiding overfitting - Best 'dropout_rate'
###########################################################

# Dropout Regularization - avoiding overfitting - (Parameter Tuning - for loop - Level 3)
# it randomly deactivates few neurons in each iteration to avoid neurons from learning similar patterns, 
#   and helps different ANN neurons learn different/independent patterns in each iteration
# It actually increased the accuracy below (rate=0.1 gave best result)

from keras.layers import Dropout

np.random.seed(100)

############################### Best 'dropout_rate'
auc1=0
auc2=0
best_auc1=best_auc1 # best auc of 1st level tuning is base accuracy at 2nd level
best_auc2=best_auc2 # best auc of 1st level tuning is base accuracy at 2nd level
best_layers = best_layers # taken from 1st level tuning
best_units = best_units # taken from 1st level tuning 
best_batch_size = best_batch_size # taken from 2nd level tuning
best_epochs = best_epochs # taken from 2nd level tuning
best_dropout_rate = 0

for i in range(10,45,5): # dropout rate (divided by 100 in code)
    classifier = Sequential()
    classifier.add(Dense(activation="relu", input_dim=11, units=best_units, kernel_initializer="uniform"))
    classifier.add(Dropout(rate=i/100)) # drops n% of neurons randomly in each iteration,instructor suggestes 0.1-0.4
    for k in range(1,best_layers): # adding hidden layers dynamically
        classifier.add(Dense(activation="relu", units=best_units, kernel_initializer="uniform"))
        classifier.add(Dropout(rate=i/100))
    classifier.add(Dense(activation="sigmoid", units=1, kernel_initializer="uniform"))
    classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    classifier.fit(X_Train, Y_Train, batch_size=best_batch_size, epochs=best_epochs)

    y_pred = classifier.predict(X_Vald1)
    y_pred = (y_pred > 0.5)
    fp_rate, tp_rate, thresholds = roc_curve(Y_Vald1, y_pred)
    auc1 = auc(fp_rate, tp_rate)

    y_pred = classifier.predict(X_Vald2)
    y_pred = (y_pred > 0.5)
    fp_rate, tp_rate, thresholds = roc_curve(Y_Vald2, y_pred)
    auc2 = auc(fp_rate, tp_rate)

    # check code progress
    print("current_auc1: ", auc1) # to check ongoing current auc1
    print("current_auc2: ", auc2) # to check ongoing current auc2
    print("current_dropout_rate: ", i/100) # to check ongoing current batch_size

    if (auc1 > best_auc1 and auc2 > best_auc2):
        best_auc1 = auc1
        best_auc2 = auc2
        best_dropout_rate = i/100
        classifier_best_2 = classifier # saving the best model

    # check ongoing best parameters during execution
    print("best_auc1: ", best_auc1) # to check ongoing best auc1
    print("best_auc2: ", best_auc2) # to check ongoing best auc2
    print("best_dropout_rate: ", best_dropout_rate) # to check ongoing best auc2
    
print("final_best_auc1: ", best_auc1)
print("final_best_auc2: ", best_auc2)
print("best_dropout_rate: ", best_dropout_rate)

classifier = classifier_best_2 # saving final model as 'classifier' to predict

# final_best_auc1:  0.715288584206
# final_best_auc2:  0.714950923113
# best_dropout_rate:  0 

# Summary: Accuracy improved at this tuning level

Epoch 1/100
7053/7053 [==============================] - 13s 2ms/step - loss: 0.4517 - acc: 0.7968
Epoch 2/100
7053/7053 [==============================] - 4s 588us/step - loss: 0.4201 - acc: 0.8238
Epoch 3/100
7053/7053 [==============================] - 4s 614us/step - loss: 0.4083 - acc: 0.8371
Epoch 4/100
7053/7053 [==============================] - 4s 510us/step - loss: 0.3965 - acc: 0.8408
Epoch 5/100
7053/7053 [==============================] - 4s 502us/step - loss: 0.3807 - acc: 0.8447
Epoch 6/100
7053/7053 [==============================] - 4s 507us/step - loss: 0.3638 - acc: 0.8537
Epoch 7/100
7053/7053 [==============================] - 4s 520us/step - loss: 0.3537 - acc: 0.8569
Epoch 8/100
7053/7053 [==============================] - 4s 506us/step - loss: 0.3469 - acc: 0.8599
Epoch 9/100
7053/7053 [==============================] - 4s 505us/step - loss: 0.3449 - acc: 0.8586
Epoch 10/100
7053/7053 [==============================] - 4s 504us/step - loss: 0.3420 - acc: 0.8613


7053/7053 [==============================] - 4s 560us/step - loss: 0.3108 - acc: 0.8742
Epoch 62/100
7053/7053 [==============================] - 4s 568us/step - loss: 0.3041 - acc: 0.8741
Epoch 63/100
7053/7053 [==============================] - 4s 606us/step - loss: 0.3059 - acc: 0.8769
Epoch 64/100
7053/7053 [==============================] - 4s 592us/step - loss: 0.3061 - acc: 0.8779
Epoch 65/100
7053/7053 [==============================] - 5s 706us/step - loss: 0.3093 - acc: 0.8723
Epoch 66/100
7053/7053 [==============================] - 5s 645us/step - loss: 0.3094 - acc: 0.8731
Epoch 67/100
7053/7053 [==============================] - 4s 609us/step - loss: 0.3043 - acc: 0.8792
Epoch 68/100
7053/7053 [==============================] - 4s 637us/step - loss: 0.3048 - acc: 0.8771
Epoch 69/100
7053/7053 [==============================] - 4s 582us/step - loss: 0.3071 - acc: 0.8742
Epoch 70/100
7053/7053 [==============================] - 4s 593us/step - loss: 0.3057 - acc: 0.8758
Epo

Epoch 18/100
7053/7053 [==============================] - 4s 564us/step - loss: 0.3453 - acc: 0.8572 0s - loss: 0.3469 - acc: 0.
Epoch 19/100
7053/7053 [==============================] - 4s 565us/step - loss: 0.3441 - acc: 0.8599
Epoch 20/100
7053/7053 [==============================] - 4s 566us/step - loss: 0.3438 - acc: 0.8596
Epoch 21/100
7053/7053 [==============================] - 4s 551us/step - loss: 0.3383 - acc: 0.8643
Epoch 22/100
7053/7053 [==============================] - 4s 578us/step - loss: 0.3379 - acc: 0.8620
Epoch 23/100
7053/7053 [==============================] - 5s 707us/step - loss: 0.3385 - acc: 0.8629
Epoch 24/100
7053/7053 [==============================] - 4s 561us/step - loss: 0.3356 - acc: 0.8654
Epoch 25/100
7053/7053 [==============================] - 4s 565us/step - loss: 0.3384 - acc: 0.8657
Epoch 26/100
7053/7053 [==============================] - 4s 583us/step - loss: 0.3400 - acc: 0.8601
Epoch 27/100
7053/7053 [==============================] - 4s 59

7053/7053 [==============================] - 5s 740us/step - loss: 0.3259 - acc: 0.8673
Epoch 78/100
7053/7053 [==============================] - 5s 772us/step - loss: 0.3317 - acc: 0.8690 0s - loss: 0.3319 -
Epoch 79/100
7053/7053 [==============================] - 5s 772us/step - loss: 0.3276 - acc: 0.8711
Epoch 80/100
7053/7053 [==============================] - 5s 725us/step - loss: 0.3277 - acc: 0.8689
Epoch 81/100
7053/7053 [==============================] - 5s 768us/step - loss: 0.3262 - acc: 0.8687
Epoch 82/100
7053/7053 [==============================] - 5s 764us/step - loss: 0.3235 - acc: 0.8683
Epoch 83/100
7053/7053 [==============================] - 5s 754us/step - loss: 0.3251 - acc: 0.8691
Epoch 84/100
7053/7053 [==============================] - 6s 860us/step - loss: 0.3261 - acc: 0.8703
Epoch 85/100
7053/7053 [==============================] - 5s 762us/step - loss: 0.3252 - acc: 0.8721
Epoch 86/100
7053/7053 [==============================] - 5s 759us/step - loss: 0.31

KeyboardInterrupt: 

In [41]:
###########################################################
############################### ANN Tuning - Level 4
############################### Best Optimizer() parameters
###########################################################

np.random.seed(100)

auc1=0
auc2=0
best_auc1=best_auc1 # from previous tuning
best_auc2=best_auc2 # from previous tuning
best_layers = best_layers # from previous tuning
best_units = best_units # from previous tuning
best_batch_size = best_batch_size # from previous tuning
best_epochs = best_epochs # from previous tuning
best_dropout_rate = best_dropout_rate # from previous tuning
best_lr = 0
best_beta_1 = 0
best_beta_2 = 0
best_decay = 0
list_1 = [0.0001,0.0003,0.001] # values taken around default values
list_2 = [0.88,0.89,0.90,0.91,0.92] # values taken around default values
list_3 = [0.96,0.97,0.98,0.99,0.996,0.997,0.998,0.999] # values taken around default values
list_4 = [0.0,0.00001] # values taken around default values
for i in list_1: # learning rate
    for j in list_2: # beta_1
            for k in list_3: # beta_2     
                            for l in list_4: # decay          
                                classifier = Sequential()
                                classifier.add(Dense(activation="relu", input_dim=11, units=best_units, kernel_initializer="uniform"))
                                for m in range(1,best_layers): # adding hidden layers dynamically
                                    classifier.add(Dense(activation="relu", units=best_units, kernel_initializer="uniform"))
                                    classifier.add(Dropout(rate=best_dropout_rate/100))
                                classifier.add(Dense(activation="sigmoid", units=1, kernel_initializer="uniform"))

                                opt = keras.optimizers.Adam(lr=i, beta_1=j, beta_2=k, decay=l)
                                # lr>=0; 0 < beta1 & beta2 < 1 but generally close to 1; decay>=0;
                                # default: opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.0)
                                # link: https://keras.io/optimizers/
                                classifier.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
                                classifier.fit(X_Train, Y_Train, batch_size=best_batch_size, epochs=best_epochs)

                                y_pred = classifier.predict(X_Vald1)
                                y_pred = (y_pred > 0.5)
                                fp_rate, tp_rate, thresholds = roc_curve(Y_Vald1, y_pred)
                                auc1 = auc(fp_rate, tp_rate)

                                y_pred = classifier.predict(X_Vald2)
                                y_pred = (y_pred > 0.5)
                                fp_rate, tp_rate, thresholds = roc_curve(Y_Vald2, y_pred)
                                auc2 = auc(fp_rate, tp_rate)

                                # check code progress
                                print("current_auc1: ", auc1) # to check ongoing current auc1
                                print("current_auc2: ", auc2) # to check ongoing current auc2
                                print("current_lr: ", i) # to check progress
                                print("current_beta_1: ", j) # to check progress
                                print("current_beta_2: ", k) # to check progress
                                print("current_decay: ", l) # to check progress

                                if (auc1 > best_auc1 and auc2 > best_auc2):
                                    best_auc1 = auc1
                                    best_auc2 = auc2
                                    best_lr = i
                                    best_beta_1 = j
                                    best_beta_2 = k
                                    best_decay = l
                                    classifier_best_4 = classifier # saving the best model

                                # check ongoing best parameters during execution
                                print("best_auc1: ", best_auc1) # to check ongoing best auc1
                                print("best_auc2: ", best_auc2) # to check ongoing best auc2
                                print("best_lr: ", best_lr) # to check ongoing best
                                print("best_beta_1: ", best_beta_1) # to check ongoing best
                                print("best_beta_2: ", best_beta_2) # to check ongoing best
                                print("best_decay: ", best_decay) # to check ongoing best
    
print("final_best_auc1: ", best_auc1)
print("final_best_auc2: ", best_auc2)
print("final_best_lr: ", best_lr)
print("final_best_beta_1: ", best_beta_1)
print("final_best_beta_2: ", best_beta_2)
print("final_best_decay: ", best_decay)

classifier = classifier_best_4 # saving final model as 'classifier' to predict

# final_best_auc1:  0.72669316286
# final_best_auc2:  0.741785463893
# final_best_lr:  0.0003
# final_best_beta_1:  0.88
# final_best_beta_2:  0.97
# final_best_decay:  1e-05

# Summary: Increased AUC significantly

Epoch 1/100
7014/7014 [==============================] - 2s 256us/step - loss: 0.5994 - acc: 0.7988
Epoch 2/100
7014/7014 [==============================] - 1s 151us/step - loss: 0.4519 - acc: 0.7993
Epoch 3/100
7014/7014 [==============================] - 1s 143us/step - loss: 0.4374 - acc: 0.7993
Epoch 4/100
7014/7014 [==============================] - 1s 169us/step - loss: 0.4314 - acc: 0.7993
Epoch 5/100
7014/7014 [==============================] - 1s 135us/step - loss: 0.4277 - acc: 0.7993
Epoch 6/100
7014/7014 [==============================] - 1s 140us/step - loss: 0.4250 - acc: 0.7993
Epoch 7/100
7014/7014 [==============================] - 1s 130us/step - loss: 0.4223 - acc: 0.8015
Epoch 8/100
7014/7014 [==============================] - 1s 143us/step - loss: 0.4199 - acc: 0.8129
Epoch 9/100
7014/7014 [==============================] - 1s 173us/step - loss: 0.4177 - acc: 0.8179
Epoch 10/100
7014/7014 [==============================] - 1s 150us/step - loss: 0.4157 - acc: 0.8252

Epoch 82/100
7014/7014 [==============================] - 1s 130us/step - loss: 0.3318 - acc: 0.8664
Epoch 83/100
7014/7014 [==============================] - 1s 128us/step - loss: 0.3311 - acc: 0.8664
Epoch 84/100
7014/7014 [==============================] - 1s 133us/step - loss: 0.3306 - acc: 0.8676
Epoch 85/100
7014/7014 [==============================] - 1s 133us/step - loss: 0.3300 - acc: 0.8666
Epoch 86/100
7014/7014 [==============================] - 1s 128us/step - loss: 0.3296 - acc: 0.8663
Epoch 87/100
7014/7014 [==============================] - 1s 131us/step - loss: 0.3296 - acc: 0.8678
Epoch 88/100
7014/7014 [==============================] - 1s 130us/step - loss: 0.3288 - acc: 0.8677
Epoch 89/100
7014/7014 [==============================] - 1s 131us/step - loss: 0.3286 - acc: 0.8663
Epoch 90/100
7014/7014 [==============================] - 1s 130us/step - loss: 0.3285 - acc: 0.8651
Epoch 91/100
7014/7014 [==============================] - 1s 127us/step - loss: 0.3279 - ac

7014/7014 [==============================] - 1s 125us/step - loss: 0.3905 - acc: 0.8433 0s - loss: 0.3791 - ac
Epoch 39/100
7014/7014 [==============================] - 1s 123us/step - loss: 0.3901 - acc: 0.8425
Epoch 40/100
7014/7014 [==============================] - 1s 123us/step - loss: 0.3897 - acc: 0.8432
Epoch 41/100
7014/7014 [==============================] - 1s 124us/step - loss: 0.3891 - acc: 0.8420
Epoch 42/100
7014/7014 [==============================] - 1s 124us/step - loss: 0.3884 - acc: 0.8437
Epoch 43/100
7014/7014 [==============================] - 1s 123us/step - loss: 0.3879 - acc: 0.8447
Epoch 44/100
7014/7014 [==============================] - 1s 126us/step - loss: 0.3872 - acc: 0.8447
Epoch 45/100
7014/7014 [==============================] - 1s 123us/step - loss: 0.3866 - acc: 0.8429
Epoch 46/100
7014/7014 [==============================] - 1s 126us/step - loss: 0.3858 - acc: 0.8460
Epoch 47/100
7014/7014 [==============================] - 1s 128us/step - loss: 0

7014/7014 [==============================] - 1s 126us/step - loss: 0.3266 - acc: 0.8684
Epoch 97/100
7014/7014 [==============================] - 1s 133us/step - loss: 0.3263 - acc: 0.8698
Epoch 98/100
7014/7014 [==============================] - 1s 147us/step - loss: 0.3261 - acc: 0.8694
Epoch 99/100
7014/7014 [==============================] - 1s 130us/step - loss: 0.3259 - acc: 0.8694
Epoch 100/100
7014/7014 [==============================] - 1s 124us/step - loss: 0.3258 - acc: 0.8688
current_auc1:  0.69956961209
current_auc2:  0.719654124796
current_lr:  0.0001
current_beta_1:  0.88
current_beta_2:  0.97
current_decay:  1e-05
best_auc1:  0.715288584206
best_auc2:  0.714950923113
best_lr:  0
best_beta_1:  0
best_beta_2:  0
best_decay:  0
Epoch 1/100
7014/7014 [==============================] - 1s 204us/step - loss: 0.6081 - acc: 0.7943
Epoch 2/100
7014/7014 [==============================] - 1s 125us/step - loss: 0.4533 - acc: 0.7993
Epoch 3/100
7014/7014 [==========================

7014/7014 [==============================] - 1s 123us/step - loss: 0.3364 - acc: 0.8647
Epoch 53/100
7014/7014 [==============================] - 1s 128us/step - loss: 0.3356 - acc: 0.8664
Epoch 54/100
7014/7014 [==============================] - 1s 127us/step - loss: 0.3347 - acc: 0.8670
Epoch 55/100
7014/7014 [==============================] - 1s 130us/step - loss: 0.3338 - acc: 0.8670 0s - loss: 0.34
Epoch 56/100
7014/7014 [==============================] - 1s 126us/step - loss: 0.3331 - acc: 0.8683
Epoch 57/100
7014/7014 [==============================] - 1s 123us/step - loss: 0.3327 - acc: 0.8676
Epoch 58/100
7014/7014 [==============================] - 1s 128us/step - loss: 0.3321 - acc: 0.8690
Epoch 59/100
7014/7014 [==============================] - 1s 127us/step - loss: 0.3313 - acc: 0.8680
Epoch 60/100
7014/7014 [==============================] - 1s 125us/step - loss: 0.3311 - acc: 0.8680
Epoch 61/100
7014/7014 [==============================] - 1s 128us/step - loss: 0.3308 -

...

Epoch 1/100
7014/7014 [==============================] - 8s 1ms/step - loss: 0.4520 - acc: 0.8017
Epoch 2/100
7014/7014 [==============================] - 2s 355us/step - loss: 0.4144 - acc: 0.8333
Epoch 3/100
7014/7014 [==============================] - 2s 352us/step - loss: 0.4031 - acc: 0.8389
Epoch 4/100
7014/7014 [==============================] - 2s 355us/step - loss: 0.3916 - acc: 0.8426
Epoch 5/100
7014/7014 [==============================] - 2s 355us/step - loss: 0.3745 - acc: 0.8486
Epoch 6/100
7014/7014 [==============================] - 2s 351us/step - loss: 0.3580 - acc: 0.8557
Epoch 7/100
7014/7014 [==============================] - 2s 352us/step - loss: 0.3479 - acc: 0.8599
Epoch 8/100
7014/7014 [==============================] - 3s 360us/step - loss: 0.3445 - acc: 0.8603
Epoch 9/100
7014/7014 [==============================] - 2s 350us/step - loss: 0.3410 - acc: 0.8643
Epoch 10/100
7014/7014 [==============================] - 2s 355us/step - loss: 0.3389 - acc: 0.8624
E

7014/7014 [==============================] - 3s 407us/step - loss: 0.2854 - acc: 0.8878
Epoch 61/100
7014/7014 [==============================] - 3s 391us/step - loss: 0.2832 - acc: 0.8885
Epoch 62/100
7014/7014 [==============================] - 3s 405us/step - loss: 0.2832 - acc: 0.8879
Epoch 63/100
7014/7014 [==============================] - 3s 466us/step - loss: 0.2827 - acc: 0.8877
Epoch 64/100
7014/7014 [==============================] - 3s 461us/step - loss: 0.2809 - acc: 0.8875
Epoch 65/100
7014/7014 [==============================] - 3s 489us/step - loss: 0.2811 - acc: 0.8874
Epoch 66/100
7014/7014 [==============================] - 3s 477us/step - loss: 0.2794 - acc: 0.8875
Epoch 67/100
7014/7014 [==============================] - 4s 564us/step - loss: 0.2770 - acc: 0.8891
Epoch 68/100
7014/7014 [==============================] - 4s 518us/step - loss: 0.2757 - acc: 0.8906 0s - loss: 0.2739
Epoch 69/100
7014/7014 [==============================] - 3s 449us/step - loss: 0.2781

7014/7014 [==============================] - 3s 462us/step - loss: 0.3320 - acc: 0.8651
Epoch 16/100
7014/7014 [==============================] - 3s 437us/step - loss: 0.3318 - acc: 0.8667
Epoch 17/100
7014/7014 [==============================] - 3s 447us/step - loss: 0.3315 - acc: 0.8695
Epoch 18/100
7014/7014 [==============================] - 4s 565us/step - loss: 0.3318 - acc: 0.8661
Epoch 19/100
7014/7014 [==============================] - 3s 476us/step - loss: 0.3294 - acc: 0.8666
Epoch 20/100
7014/7014 [==============================] - 4s 525us/step - loss: 0.3260 - acc: 0.8688
Epoch 21/100
7014/7014 [==============================] - 3s 442us/step - loss: 0.3286 - acc: 0.8671 0s - loss: 0.3249 -
Epoch 22/100
7014/7014 [==============================] - 4s 512us/step - loss: 0.3260 - acc: 0.8663
Epoch 23/100
7014/7014 [==============================] - 4s 514us/step - loss: 0.3250 - acc: 0.8664
Epoch 24/100
7014/7014 [==============================] - 4s 503us/step - loss: 0.32

7014/7014 [==============================] - 3s 453us/step - loss: 0.2686 - acc: 0.8941
Epoch 74/100
7014/7014 [==============================] - 3s 436us/step - loss: 0.2688 - acc: 0.8921
Epoch 75/100
7014/7014 [==============================] - 3s 445us/step - loss: 0.2675 - acc: 0.8929
Epoch 76/100
7014/7014 [==============================] - 3s 441us/step - loss: 0.2666 - acc: 0.8926
Epoch 77/100
7014/7014 [==============================] - 3s 432us/step - loss: 0.2645 - acc: 0.8965
Epoch 78/100
7014/7014 [==============================] - 3s 435us/step - loss: 0.2643 - acc: 0.8964
Epoch 79/100
7014/7014 [==============================] - 3s 447us/step - loss: 0.2629 - acc: 0.8951
Epoch 80/100
7014/7014 [==============================] - 3s 438us/step - loss: 0.2622 - acc: 0.8959 1s - lo
Epoch 81/100
7014/7014 [==============================] - 3s 448us/step - loss: 0.2607 - acc: 0.8992
Epoch 82/100
7014/7014 [==============================] - 3s 436us/step - loss: 0.2616 - acc: 0.

KeyboardInterrupt: 

In [45]:
# Calculating auc for Vald1 and Vald2 sets

# select best classifiers among 'classifier_best', 'classifier_best_1', 'classifier_best_2'
# depending on which tuning level above gave best model
classifier = classifier_best_4

# Predict for Vald1 set
y_pred = classifier.predict(X_Vald1)
y_pred = (y_pred > 0.5)

# creating CF matrix
from sklearn.metrics import confusion_matrix
cm1 = confusion_matrix(y_pred, Y_Vald1)

# calculating auc
from sklearn.metrics import roc_curve, auc, roc_auc_score
fp_rate, tp_rate, thresholds = roc_curve(Y_Vald1, y_pred)
print(auc(fp_rate, tp_rate))

# printing CF
## cm

###########################################################

# Predict for Vald2 set
y_pred = classifier.predict(X_Vald2)
y_pred = (y_pred > 0.5)

# creating CF matrix
from sklearn.metrics import confusion_matrix
cm2 = confusion_matrix(y_pred, Y_Vald2)

# calculating auc
from sklearn.metrics import roc_curve, auc, roc_auc_score
fp_rate, tp_rate, thresholds = roc_curve(Y_Vald2, y_pred)
print(auc(fp_rate, tp_rate))

# printing CF
## cm

###########################################################

0.72669316286
0.741785463893


In [46]:
#### Testing final accuracy(auc) on Test set (only once at the end)

# select best classifiers among 'classifier_best', 'classifier_best_1', 'classifier_best_2'
# depending on which tuning level above gave best model
classifier = classifier_best_4

# Predict for Test set
y_pred = classifier.predict(X_Test)
y_pred = (y_pred > 0.50)

# creating CF matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred, Y_Test)

# calculating auc
from sklearn.metrics import roc_curve, auc, roc_auc_score
fp_rate, tp_rate, thresholds = roc_curve(Y_Test, y_pred)
print(auc(fp_rate, tp_rate))

# printing CF
## cm

0.745833241291


In [47]:
cm

array([[767, 101],
       [ 45, 122]], dtype=int64)

In [ ]:
# K-Fold Cross Validation - avoid overfitting

# Video 30 - https://www.udemy.com/deeplearning/learn/v4/t/lecture/6743752?start=0
# but approach of multiple CV sets is equivalent to k-fold CV

In [355]:
# Grid Search - Parameter Tuning (involved K-Fold CV)

# Video 31 - https://www.udemy.com/deeplearning/learn/v4/t/lecture/6743752?start=0
# but 'for loop' based parameter tuning is as good as 'grid search'

### 7. Download and Upload Model

In [53]:
###########################################################
############################### Download and Upload Keras Model
###########################################################

#### Download Keras Model
from keras.models import load_model
# classifier_best_4.save('Churn_ANN_Final_Model.h5') # default save: saves in the code folder
classifier_best_4.save('C:/Users/anurag.trivedi/3. Saved Models/Churn_ANN_Final_Model1.h5')

In [54]:
#### Upload Keras Model
classifier_best_x = load_model('C:/Users/anurag.trivedi/3. Saved Models/Churn_ANN_Final_Model1.h5')

### Summary Notes:

In [ ]:
# Summary Notes:
# Confusion matrix generated here is very different from the one generated using Xgboost in R

# ANN(Python) optimized for AUC:
# array([[767, 101],
#        [ 45, 122]], dtype=int64)
# AUC:0.7458 Accuracy: 85.89% - equivalent to one of the best obtained by the Udemy course instructor

# Xgboost optimized for AUC:
# Final CF matrix of Python ANN:
#       0   1
#   0 764 184
#   1   2  29
# AUC:0.87 Accuracy: 81.0%

# Xgboost optimized for F-Score:
# Final CF matrix of Python ANN:
#       0   1
#   0 740 113
#   1 26  100
# AUC:0.83 Accuracy: 85.8%

# Best AUC of ANN is 0.73 but best AUC of Xgboost is 0.87; but CF matrix of ANN looks much more balanced than Xgboost
# ANN takes much more time than Xgboost. 'ANN 1 Night' = 'Xgboost 1 Hr'
# Best models seem similar for both Xgboost(F-Score optimized) and ANN(AUC optimized)
# Recommendation - For smaller tabular data(7000 in this case), go for Xgboost and not ANN